# This file is oriented to process the SOCC following the main purpose of "comment-moderation' project.


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('raw/SFU_constructiveness_toxicity_corpus.csv')
df_backup = pd.read_csv('raw/SFU_constructiveness_toxicity_corpus.csv')

In [4]:
goal = df[['comment_text', 'toxicity_level', 'toxicity_level:confidence']]

In [5]:
goal

,comment_text,toxicity_level,toxicity_level:confidence
0,"While technology does march on, sometimes it t...",1,1
1,Everyone is still missing the point of what th...,1\n2,0.9111\n0.0889
2,You've never used an Apple product have you?,1\n2,0.6308\n0.2937
3,You may be using a blackberry. I'm still using...,1\n2,0.7346\n0.2023
4,"Of course we all know, unfortunately, there is...",2\n1,0.4429\n0.3066
...,...,...,...
1038,Hillary Clinton wasn't defeated for being a wo...,2\n1,0.671\n0.329
1039,Very well said... and thank you for shouting i...,2\n1,0.671\n0.329
1040,And now that we've heard Heneins views as a wo...,1,1
1041,I will re-read this in the months or years to ...,1,1


In [6]:
goal.columns = ['text', 'label', 'confidence']

In [7]:
goal

,text,label,confidence
0,"While technology does march on, sometimes it t...",1,1
1,Everyone is still missing the point of what th...,1\n2,0.9111\n0.0889
2,You've never used an Apple product have you?,1\n2,0.6308\n0.2937
3,You may be using a blackberry. I'm still using...,1\n2,0.7346\n0.2023
4,"Of course we all know, unfortunately, there is...",2\n1,0.4429\n0.3066
...,...,...,...
1038,Hillary Clinton wasn't defeated for being a wo...,2\n1,0.671\n0.329
1039,Very well said... and thank you for shouting i...,2\n1,0.671\n0.329
1040,And now that we've heard Heneins views as a wo...,1,1
1041,I will re-read this in the months or years to ...,1,1


In [8]:
goal = goal[['text', 'confidence', 'label']]
goal

,text,confidence,label
0,"While technology does march on, sometimes it t...",1,1
1,Everyone is still missing the point of what th...,0.9111\n0.0889,1\n2
2,You've never used an Apple product have you?,0.6308\n0.2937,1\n2
3,You may be using a blackberry. I'm still using...,0.7346\n0.2023,1\n2
4,"Of course we all know, unfortunately, there is...",0.4429\n0.3066,2\n1
...,...,...,...
1038,Hillary Clinton wasn't defeated for being a wo...,0.671\n0.329,2\n1
1039,Very well said... and thank you for shouting i...,0.671\n0.329,2\n1
1040,And now that we've heard Heneins views as a wo...,1,1
1041,I will re-read this in the months or years to ...,1,1


In [9]:
good = goal.loc[~goal['label'].astype(str).str.contains('\n')]
print('Good:')
display(good)

bad = goal.loc[goal['label'].astype(str).str.contains('\n')]
print('Bad:')
display(bad)

Good:


,text,confidence,label
0,"While technology does march on, sometimes it t...",1,1
9,Time for the elders and chiefs to stand up to ...,1,1
16,Playing the race card.,1,1
24,While I support the notion of subsidized dayca...,1,1
54,----'A Trump administration might make it hard...,1,1
...,...,...,...
1034,"As a lawyer, I have nothing but the highest re...",1,1
1035,"And with hindsight being what it is, lets just...",1,1
1040,And now that we've heard Heneins views as a wo...,1,1
1041,I will re-read this in the months or years to ...,1,1


Bad:


,text,confidence,label
1,Everyone is still missing the point of what th...,0.9111\n0.0889,1\n2
2,You've never used an Apple product have you?,0.6308\n0.2937,1\n2
3,You may be using a blackberry. I'm still using...,0.7346\n0.2023,1\n2
4,"Of course we all know, unfortunately, there is...",0.4429\n0.3066,2\n1
5,Simpson claims the budget surplus isn't a surp...,0.5102\n0.2344,1\n2
...,...,...,...
1031,"The writer is big on dogma and makeup, but wea...",0.675\n0.325,2\n1
1036,'What are you going to tell your daughters?'Te...,0.671\n0.329,2\n1
1037,There is something inherently sexist about the...,0.671\n0.329,2\n1
1038,Hillary Clinton wasn't defeated for being a wo...,0.671\n0.329,2\n1


In [10]:
import numpy as np

# Process bad before.
okay = pd.DataFrame(columns=bad.columns)

for row in bad.itertuples():
    label = row[3]
    confidence = row[2]
    labels = label.split('\n')
    confidences = confidence.split('\n')

    if  set(labels[:2]) == {'2', '3'} or \
        set(labels[:2]) == {'2', '4'} or \
        set(labels[:2]) == {'3', '4'}:
        new_confidence = max(float(confidences[0]), float(confidences[1]))
        new_row = pd.DataFrame({
            'text': [row[1]],
            'confidence': [str(new_confidence)],
            'label': ['toxic'],
        })
        okay = pd.concat([okay, new_row], ignore_index=True)
    else: 
        for l, c in zip(labels, confidences):
            la = 'normal'
            if l == '3' or l == '4' or l == '2':
                la = 'toxic'
            
            new_row = pd.DataFrame({
                'text': [row[1]],
                'confidence': [c],
                'label': [la],
            })
            okay = pd.concat([okay, new_row], ignore_index=True)
        

In [11]:
# Push all good ones into okay
new_good = good.copy()
new_good.loc[(new_good['label'] == '1'), 'label'] = 'normal'
new_good.loc[(new_good['label'] == '3') | (new_good['label'] == '4') | (new_good['label'] == '2'), 'label'] = 'toxic'
okay = pd.concat([okay, new_good], ignore_index=True)
okay

,text,confidence,label
0,Everyone is still missing the point of what th...,0.9111,normal
1,Everyone is still missing the point of what th...,0.0889,toxic
2,You've never used an Apple product have you?,0.6308,normal
3,You've never used an Apple product have you?,0.2937,toxic
4,You may be using a blackberry. I'm still using...,0.7346,normal
...,...,...,...
1620,"As a lawyer, I have nothing but the highest re...",1,normal
1621,"And with hindsight being what it is, lets just...",1,normal
1622,And now that we've heard Heneins views as a wo...,1,normal
1623,I will re-read this in the months or years to ...,1,normal


In [12]:
# Finally, I will export it into csv

In [27]:
okay.to_csv('processed/socc_method2.csv', index=False)